# This is the code which solves the Static Traffic Assignment Problem

In [3]:
import numpy as np

In [4]:
network = 'data/Anaheim'

## 1. We load the graph and the demand
Both graph and demand are in csv file

In [5]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

In [6]:
# in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix
try:
    demand.shape[1]
except:
    demand = np.array([demand])
nb_ods = int(demand.shape[0])

Then store the links in a dictionary

In [7]:
# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])

Then, we define the function which gives the travel time as a function of the flow

In [8]:
def travel_time(f):
    return graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)

In [9]:
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)

## 2. We compute the all or nothing flow allocation

In [10]:
import scipy

In [11]:
from scipy.sparse.csgraph import dijkstra

To use the Dijkstra's algorithm class of scipy we need to define the adjacent matrix of the graph

In [12]:
def update_travel_time(tt):
    for i in range(graph.shape[0]):
        G[int(graph[i][1])][int(graph[i][2])] = tt[i]

In [13]:
G = np.zeros(shape=(nb_nodes,nb_nodes))
update_travel_time(travel_time(np.zeros(nb_links)))
print(G)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Now let's compute the all or nothing allocation

In [14]:
# computing the all or nothing flow
def all_or_nothing():
    # using scipy to compute dijkstra
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True)
    faon = np.zeros(shape = nb_links)
    for i in range(nb_ods):
        o_tmp = int(demand[i][0])
        d_tmp = int(demand[i][1])
        flow_tmp = demand[i][2]

        node_tmp_d = d_tmp
        while node_tmp_d != o_tmp:
            node_tmp = return_predecessors[o_tmp][node_tmp_d]
            link_tmp = int(graph_dict[node_tmp][node_tmp_d])
            faon[link_tmp] += flow_tmp
            node_tmp_d = node_tmp
    return faon

We define the line search

In [15]:
def potential(graph, f):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    links = int(np.max(graph[:, 0]) + 1)
    g = np.copy(
        graph.dot(np.diag([1., 1., 1., 1., 1 / 2., 1 / 3., 1 / 4., 1 / 5.])))
    x = np.power(f.reshape((links, 1)), np.array([1, 2, 3, 4, 5]))
    return np.sum(np.einsum('ij,ij->i', x, g[:, 3:]))


def line_search(f, res=20):
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

Now let run the Frank-Wolf's algorithm with a line search to find alpha

In [16]:
eps=1e-8
f = all_or_nothing()
update_travel_time(travel_time(f))

for i in range(100):
    if(i%10==0):
        print(i)
    faon = all_or_nothing() 
    s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon))
    if s < eps:
        break
    f = (1. - s) * f + s * faon
    update_travel_time(travel_time(f))

0
10
20
30
40
50
60
70
80
90


In [17]:
print(G)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
print(f)

[7.07490000e+03 9.66250000e+03 7.66900000e+03 1.21738000e+04
 3.10431426e+03 6.57660000e+03 7.13710000e+03 7.22100000e+02
 1.12943939e+03 1.10828966e+03 4.58673222e+02 7.09583723e+01
 4.85800000e+02 4.88200000e+02 3.70000000e+01 1.25200000e+02
 4.07100000e+02 2.49000000e+02 6.48300000e+02 1.57044902e+03
 1.33463934e+03 7.68578883e+02 5.30505901e+02 5.03600000e+02
 1.49129555e+03 1.40807082e+03 1.40214208e+03 8.21976194e+02
 1.52250000e+03 3.38070299e+03 3.27507853e+03 4.28278387e+03
 4.64502348e+03 3.15636530e+03 1.81564933e+03 3.42007467e+03
 3.21954209e+03 3.95183359e+03 3.46785013e+03 3.70082550e+03
 3.82210960e+03 1.92436697e+03 2.66802230e+03 3.02775724e+03
 2.78971629e+03 2.97513325e+03 2.55609662e+03 3.51720664e+03
 2.95916854e+03 3.49929071e+03 3.15533996e+03 2.85802552e+03
 2.07244756e+03 2.18895080e+03 1.58089711e+03 4.02927711e+03
 7.60409419e+02 3.13848980e+03 1.40891143e+03 8.60865563e+02
 7.96618468e+02 8.29480099e+02 1.04612806e+03 7.14460986e+02
 4.57535721e+02 9.218413